###Install depencencies

In [ ]:
!pip install -q -U diffusers transformers accelerate

###Import SVD pipeline

In [ ]:
import torch

from diffusers import StableVideoDiffusionPipeline
from diffusers.utils import load_image, export_to_video

###Setup SVD pipeline

In [ ]:
pipe = StableVideoDiffusionPipeline.from_pretrained(
    "stabilityai/stable-video-diffusion-img2vid-xt", torch_dtype=torch.float16, variant="fp16"
)
pipe.enable_model_cpu_offload()

###Upload image

In [ ]:
image = load_image("./surf.jpg")
image = image.resize((1024, 576))
image

###First run pipeline

In [ ]:
generator = torch.manual_seed(42)
frames = pipe(image, decode_chunk_size=8, generator=generator).frames[0]
export_to_video(frames, "output.mp4", fps=7)

###Run optimized pipeline (less GPU RAM is used, but we sacrifice a little quality)

In [ ]:
pipe.unet.enable_forward_chunking()
frames = pipe(
    image,
    decode_chunk_size=2,
    generator=generator,
    num_frames=25,
    motion_bucket_id=180,
    noise_aug_strength=0.1
  ).frames[0]
export_to_video(frames, "output.mp4", fps=7)